In [2]:
import pandas as pd

# # Load dataset
# df = pd.read_csv("../../datasets/ieee-train-merged_imputed_cleaned_dropped_stuff.csv")

# # Sort dataset by time
# df = df.sort_values(by="TransactionDT").reset_index(drop=True)

# # Create time-based features
# df['TimeInDay'] = df.TransactionDT % 86400
# df['Cents'] = df.TransactionAmt % 1

# # One-hot encoding for categorical features
# df = pd.get_dummies(df, columns=['ProductCD', 'card4', 'card6', 'P_emaildomain'], drop_first=True)


In [3]:
# # Define split ratios
# train_size = 0.65
# valid_size = 0.15
# test_size = 0.20

# # Compute split indices
# train_index = int(len(df) * train_size)
# valid_index = int(len(df) * (train_size + valid_size))

# # Split dataset
# df_train = df.iloc[:train_index]
# df_valid = df.iloc[train_index:valid_index]
# df_test = df.iloc[valid_index:]

# Define features and target
X_train = pd.read_csv("/Users/adityasingh/Desktop/DESKTOP/Reranked-Reasoned-Retrieval-Augmented-Financial-Fraud-Detection_Retard/IEEE-CIS/IEEE-CIS-CORRECTED/train_df.csv")
y_train = pd.read_csv("/Users/adityasingh/Desktop/DESKTOP/Reranked-Reasoned-Retrieval-Augmented-Financial-Fraud-Detection_Retard/IEEE-CIS/IEEE-CIS-CORRECTED/train_y_df.csv")

X_valid = pd.read_csv("/Users/adityasingh/Desktop/DESKTOP/Reranked-Reasoned-Retrieval-Augmented-Financial-Fraud-Detection_Retard/IEEE-CIS/IEEE-CIS-CORRECTED/val_df.csv")
y_valid = pd.read_csv("/Users/adityasingh/Desktop/DESKTOP/Reranked-Reasoned-Retrieval-Augmented-Financial-Fraud-Detection_Retard/IEEE-CIS/IEEE-CIS-CORRECTED/val_y_df.csv")

X_test = pd.read_csv("/Users/adityasingh/Desktop/DESKTOP/Reranked-Reasoned-Retrieval-Augmented-Financial-Fraud-Detection_Retard/IEEE-CIS/IEEE-CIS-CORRECTED/test_df.csv")
y_test = pd.read_csv("/Users/adityasingh/Desktop/DESKTOP/Reranked-Reasoned-Retrieval-Augmented-Financial-Fraud-Detection_Retard/IEEE-CIS/IEEE-CIS-CORRECTED/test_y_df.csv")


FileNotFoundError: [Errno 2] No such file or directory: '/Users/adityasingh/Desktop/DESKTOP/Reranked-Reasoned-Retrieval-Augmented-Financial-Fraud-Detection_Retard/IEEE-CIS/IEEE-CIS-CORRECTED/train_df.csv'

In [8]:
from xgboost import XGBClassifier

# Initialize XGBoost model
xgb_model = XGBClassifier(
    n_estimators=200,    # Number of trees
    max_depth=6,         # Limit depth to avoid overfitting
    learning_rate=0.05,  # Step size shrinkage
    subsample=0.8,       # Randomly sample 80% of data
    colsample_bytree=0.8, # Use 80% of features per tree
    scale_pos_weight=10,  # Handle class imbalance (adjust based on fraud ratio)
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss' # Log loss is suitable for classification
)

# Train the model
xgb_model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=True)


XGBoostError: 
XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ["dlopen(/Users/adityasingh/Desktop/DESKTOP/Reranked-Reasoned-Retrieval-Augmented-Financial-Fraud-Detection_Retard/.venv/lib/python3.12/site-packages/xgboost/lib/libxgboost.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib\n  Referenced from: <89AD948E-E564-3266-867D-7AF89D6488F0> /Users/adityasingh/Desktop/DESKTOP/Reranked-Reasoned-Retrieval-Augmented-Financial-Fraud-Detection_Retard/.venv/lib/python3.12/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: tried: '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libo

In [3]:
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report, confusion_matrix, average_precision_score

# Make predictions
y_valid_pred = xgb_model.predict(X_valid)
y_test_pred = xgb_model.predict(X_test)

# Probability scores for AUC computation
y_valid_prob = xgb_model.predict_proba(X_valid)[:, 1]
y_test_prob = xgb_model.predict_proba(X_test)[:, 1]

# AUC Scores
valid_auc = roc_auc_score(y_valid, y_valid_prob)
test_auc = roc_auc_score(y_test, y_test_prob)

# AUCPR Scores
valid_aucpr = average_precision_score(y_valid, y_valid_prob)
test_aucpr = average_precision_score(y_test, y_test_prob)

# Print results
print(f"Validation AUC: {valid_auc:.4f}")
print(f"Test AUC: {test_auc:.4f}")
print(f"Validation AUCPR: {valid_aucpr:.4f}")
print(f"Test AUCPR: {test_aucpr:.4f}")

# Classification Reports
print("Validation Classification Report:\n", classification_report(y_valid, y_valid_pred))
print("Test Classification Report:\n", classification_report(y_test, y_test_pred))

# Confusion Matrix
print("Confusion Matrix (Test Set):")
print(confusion_matrix(y_test, y_test_pred))


NameError: name 'xgb_model' is not defined

In [8]:
from sklearn.model_selection import GridSearchCV

# Define hyperparameter grid
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [4, 6, 8],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# Grid Search
grid_search = GridSearchCV(XGBClassifier(scale_pos_weight=10, random_state=42, use_label_encoder=False, eval_metric='logloss'), 
                           param_grid, scoring='roc_auc', cv=3, n_jobs=-1)

grid_search.fit(X_train, y_train)

# Best hyperparameters
print("Best Parameters:", grid_search.best_params_)


/Users/annimukh/miniconda3/envs/fraud-detec/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [02:29:04] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/annimukh/miniconda3/envs/fraud-detec/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [02:29:04] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/annimukh/miniconda3/envs/fraud-detec/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [02:29:04] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/annimukh/miniconda3/envs/fraud-detec/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [02:29:04] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters

Best Parameters: {'colsample_bytree': 1.0, 'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 200, 'subsample': 0.8}


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Get feature importances
importances = xgb_model.feature_importances_
feature_names = np.array(X_train.columns)

# Sort and plot
sorted_idx = np.argsort(importances)[::-1][:20]  # Top 20 features
plt.figure(figsize=(10, 6))
plt.barh(feature_names[sorted_idx], importances[sorted_idx])
plt.xlabel("Feature Importance")
plt.title("Top 20 Important Features (XGBoost)")
plt.gca().invert_yaxis()
plt.show()
